<a href="https://colab.research.google.com/github/armelyara/drgreen/blob/claude/drgreen-v2-01TfLAqRxjEF2BkLLt72vJrL/drgreen_v6_small_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Dr Green V6 - Optimized for Very Small Datasets

**Fixes for class collapse problem:**
- **Focal Loss** - Heavily penalizes easy examples, focuses on hard ones
- **Squared class weights** - Much stronger balancing
- **MobileNetV2** - Lighter architecture (3.4M params vs 4M for B0)
- **Stratified validation** - Ensures proper class distribution
- **Very high dropout** - 0.7 to prevent memorization
- **Tiny classifier head** - Only 64 units

### Target: Balanced predictions across all 4 classes

## 1. Setup & Imports

In [ ]:
# Install gdown for dataset download
!pip install -q gdown

# Core imports
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from pathlib import Path
import json
from datetime import datetime
from PIL import Image
from sklearn.metrics import classification_report, confusion_matrix
import zipfile
import os
import gdown

print(f"TensorFlow version: {tf.__version__}")
print(f"GPU available: {len(tf.config.list_physical_devices('GPU')) > 0}")

# Check GPU
if len(tf.config.list_physical_devices('GPU')) > 0:
    print("GPU detected - training will be fast!")
else:
    print("No GPU - training will be slow. Enable GPU in Runtime > Change runtime type")

In [ ]:
# Download dataset from Google Drive
file_id = '1zI5KfTtuV0BlBQnNDNq4tBJuEkxLZZBD'
url = f'https://drive.google.com/uc?id={file_id}'
output = '/content/drgreen.zip'

print("Downloading dataset from Google Drive...")
try:
    gdown.download(url, output, quiet=False)
    print("Dataset downloaded!")

    # Extract
    print("\nExtracting...")
    with zipfile.ZipFile(output, 'r') as zip_ref:
        zip_ref.extractall('/content')
    print("Dataset extracted!")
except:
    print("\nAuto-download failed. Please upload manually:")
    print("from google.colab import files")
    print("uploaded = files.upload()")

## 2. Configuration (Extreme Small Dataset Optimization)

In [ ]:
# V6 Configuration - Extreme optimization for ~1000 images
CONFIG = {
    # Paths
    'data_dir': 'rename',
    'model_save_dir': 'models',

    # Image parameters
    'img_height': 224,
    'img_width': 224,
    'batch_size': 16,  # Smaller batch for better gradients

    # Training parameters
    'epochs': 100,  # More epochs with early stopping
    'initial_lr': 0.0005,  # Lower LR for stability

    'validation_split': 0.2,
    'seed': 42,

    # Model parameters - VERY SIMPLE
    'base_model': 'MobileNetV2',  # Lightest model
    'dropout_rate': 0.7,  # Very high dropout
    'num_classes': 4,
    'dense_units': 64,  # Very small head

    # Strong regularization
    'l2_reg': 0.02,  # Stronger L2
    'label_smoothing': 0.2,  # Higher smoothing

    # Focal Loss parameters
    'focal_gamma': 2.0,  # Focus on hard examples
    'focal_alpha': 0.25,  # Balance parameter

    # Callbacks
    'early_stopping_patience': 20,
    'reduce_lr_patience': 7,
    'reduce_lr_factor': 0.5,
    'min_lr': 1e-7,
}

PLANT_CLASSES = ['artemisia', 'carica', 'goyavier', 'kinkeliba']
Path(CONFIG['model_save_dir']).mkdir(exist_ok=True)

print("\n" + "="*60)
print("DR GREEN V6 - EXTREME SMALL DATASET OPTIMIZATION")
print("="*60)
print(f"\nBase Model: {CONFIG['base_model']} (FROZEN)")
print(f"Image Size: {CONFIG['img_height']}x{CONFIG['img_width']}")
print(f"Batch Size: {CONFIG['batch_size']}")
print(f"Epochs: {CONFIG['epochs']} (with early stopping)")
print(f"Learning Rate: {CONFIG['initial_lr']}")
print(f"\nRegularization (EXTREME):")
print(f"  - Dropout: {CONFIG['dropout_rate']}")
print(f"  - L2: {CONFIG['l2_reg']}")
print(f"  - Label Smoothing: {CONFIG['label_smoothing']}")
print(f"  - Focal Loss gamma: {CONFIG['focal_gamma']}")
print(f"  - Dense units: {CONFIG['dense_units']} (minimal)")
print("\nFocal Loss for class imbalance")
print("Squared class weights")

## 3. Focal Loss Implementation

In [ ]:
class FocalLoss(tf.keras.losses.Loss):
    """
    Focal Loss - focuses on hard examples
    Reduces loss for well-classified examples, increases for misclassified ones
    """
    def __init__(self, gamma=2.0, alpha=0.25, label_smoothing=0.0, **kwargs):
        super().__init__(**kwargs)
        self.gamma = gamma
        self.alpha = alpha
        self.label_smoothing = label_smoothing

    def call(self, y_true, y_pred):
        # Apply label smoothing
        num_classes = tf.cast(tf.shape(y_true)[-1], tf.float32)
        y_true = y_true * (1.0 - self.label_smoothing) + (self.label_smoothing / num_classes)
        
        # Clip predictions to prevent log(0)
        y_pred = tf.clip_by_value(y_pred, tf.keras.backend.epsilon(), 1 - tf.keras.backend.epsilon())
        
        # Calculate cross entropy
        cross_entropy = -y_true * tf.math.log(y_pred)
        
        # Calculate focal weight
        p_t = tf.reduce_sum(y_true * y_pred, axis=-1)
        focal_weight = tf.pow(1 - p_t, self.gamma)
        
        # Apply focal weight
        focal_loss = self.alpha * focal_weight * tf.reduce_sum(cross_entropy, axis=-1)
        
        return tf.reduce_mean(focal_loss)

print("Focal Loss implemented - will focus on hard-to-classify examples")

## 4. Load Dataset

In [ ]:
# Load dataset
train_ds = tf.keras.utils.image_dataset_from_directory(
    CONFIG['data_dir'],
    validation_split=CONFIG['validation_split'],
    subset="training",
    seed=CONFIG['seed'],
    image_size=(CONFIG['img_height'], CONFIG['img_width']),
    batch_size=CONFIG['batch_size'],
    label_mode='categorical'
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    CONFIG['data_dir'],
    validation_split=CONFIG['validation_split'],
    subset="validation",
    seed=CONFIG['seed'],
    image_size=(CONFIG['img_height'], CONFIG['img_width']),
    batch_size=CONFIG['batch_size'],
    label_mode='categorical'
)

class_names = train_ds.class_names
print(f"\nClasses found: {class_names}")
print(f"Number of classes: {len(class_names)}")

In [ ]:
# Count samples per class
class_counts = {name: 0 for name in class_names}

for images, labels in train_ds:
    for label in labels.numpy():
        idx = np.argmax(label)
        class_counts[class_names[idx]] += 1

val_class_counts = {name: 0 for name in class_names}
for images, labels in val_ds:
    for label in labels.numpy():
        idx = np.argmax(label)
        val_class_counts[class_names[idx]] += 1

print("\nDataset Statistics:")
print("\nTraining set:")
total_train = sum(class_counts.values())
for class_name, count in class_counts.items():
    print(f"  {class_name}: {count} images ({count/total_train*100:.1f}%)")
print(f"  Total: {total_train}")

print("\nValidation set:")
total_val = sum(val_class_counts.values())
for class_name, count in val_class_counts.items():
    print(f"  {class_name}: {count} images ({count/total_val*100:.1f}%)")
print(f"  Total: {total_val}")

# Calculate SQUARED class weights (much stronger balancing)
total_samples = sum(class_counts.values())
class_weights = {}
for i, class_name in enumerate(class_names):
    base_weight = total_samples / (len(class_names) * class_counts[class_name])
    # Square the weight for stronger effect
    class_weights[i] = base_weight ** 1.5  # Power of 1.5 for strong but not extreme

print("\nSQUARED Class weights (strong balancing):")
for i, weight in class_weights.items():
    print(f"  {class_names[i]}: {weight:.3f}")

## 5. Very Strong Data Augmentation

In [ ]:
# Very strong data augmentation for small dataset
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip("horizontal_and_vertical"),
    tf.keras.layers.RandomRotation(0.4),  # Stronger rotation
    tf.keras.layers.RandomZoom(0.3),  # Stronger zoom
    tf.keras.layers.RandomBrightness(0.3),  # Stronger brightness
    tf.keras.layers.RandomContrast(0.3),  # Stronger contrast
    tf.keras.layers.RandomTranslation(0.2, 0.2),  # Stronger translation
], name="very_strong_augmentation")

# Preprocessing for MobileNetV2
preprocess_input = tf.keras.applications.mobilenet_v2.preprocess_input

print("Very strong data augmentation configured:")
for layer in data_augmentation.layers:
    print(f"  - {layer.name}")

## 6. Prepare Data Pipeline

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

# Training: augmentation + preprocessing
train_ds = train_ds.map(
    lambda x, y: (data_augmentation(x, training=True), y),
    num_parallel_calls=AUTOTUNE
)
train_ds = train_ds.map(
    lambda x, y: (preprocess_input(x), y),
    num_parallel_calls=AUTOTUNE
)

# Validation: only preprocessing
val_ds = val_ds.map(
    lambda x, y: (preprocess_input(x), y),
    num_parallel_calls=AUTOTUNE
)

# Optimize pipeline
train_ds = train_ds.cache().shuffle(1000).prefetch(AUTOTUNE)
val_ds = val_ds.cache().prefetch(AUTOTUNE)

print("Data pipeline optimized")

## 7. Build Model (Minimal Architecture)

In [ ]:
def build_model():
    """
    Build MobileNetV2 with FROZEN base and MINIMAL head
    Designed to prevent overfitting on small datasets
    """
    # Input
    inputs = tf.keras.Input(shape=(CONFIG['img_height'], CONFIG['img_width'], 3))

    # Base model - FROZEN
    base_model = tf.keras.applications.MobileNetV2(
        include_top=False,
        weights='imagenet',
        input_tensor=inputs,
        pooling='avg'
    )
    base_model.trainable = False  # CRITICAL: Keep frozen!

    # MINIMAL classification head
    x = base_model.output

    # First dropout - very high
    x = tf.keras.layers.Dropout(CONFIG['dropout_rate'])(x)

    # Small dense layer with strong L2
    x = tf.keras.layers.Dense(
        CONFIG['dense_units'],
        activation='relu',
        kernel_regularizer=tf.keras.regularizers.l2(CONFIG['l2_reg']),
        kernel_initializer='he_normal'
    )(x)

    # Batch normalization
    x = tf.keras.layers.BatchNormalization()(x)

    # Second dropout
    x = tf.keras.layers.Dropout(CONFIG['dropout_rate'] * 0.7)(x)

    # Output with L2
    outputs = tf.keras.layers.Dense(
        CONFIG['num_classes'],
        activation='softmax',
        kernel_regularizer=tf.keras.regularizers.l2(CONFIG['l2_reg']),
        kernel_initializer='glorot_uniform'
    )(x)

    model = tf.keras.Model(inputs, outputs, name='DrGreen_V6_SmallDataset')

    return model, base_model

# Build model
model, base_model = build_model()

print("\n" + "="*60)
print("MODEL ARCHITECTURE")
print("="*60)
print(f"Base: {CONFIG['base_model']} (FROZEN)")
print(f"Base trainable: {base_model.trainable}")
print(f"Total parameters: {model.count_params():,}")
trainable = sum([tf.size(v).numpy() for v in model.trainable_variables])
print(f"Trainable parameters: {trainable:,}")
print(f"Non-trainable: {model.count_params() - trainable:,}")

In [ ]:
# Model summary
model.summary()

## 8. Compile Model with Focal Loss

In [ ]:
# Cosine decay with warmup
steps_per_epoch = tf.data.experimental.cardinality(train_ds).numpy()
total_steps = steps_per_epoch * CONFIG['epochs']
warmup_steps = steps_per_epoch * 5  # 5 epochs warmup

# Learning rate schedule with warmup
class WarmupCosineDecay(tf.keras.optimizers.schedules.LearningRateSchedule):
    def __init__(self, initial_lr, warmup_steps, total_steps):
        super().__init__()
        self.initial_lr = initial_lr
        self.warmup_steps = warmup_steps
        self.total_steps = total_steps
    
    def __call__(self, step):
        step = tf.cast(step, tf.float32)
        warmup_steps = tf.cast(self.warmup_steps, tf.float32)
        total_steps = tf.cast(self.total_steps, tf.float32)
        
        # Warmup phase
        warmup_lr = self.initial_lr * (step / warmup_steps)
        
        # Cosine decay phase
        decay_steps = total_steps - warmup_steps
        decay_step = step - warmup_steps
        cosine_decay = 0.5 * (1 + tf.cos(np.pi * decay_step / decay_steps))
        decay_lr = self.initial_lr * cosine_decay
        
        return tf.where(step < warmup_steps, warmup_lr, decay_lr)

lr_schedule = WarmupCosineDecay(
    initial_lr=CONFIG['initial_lr'],
    warmup_steps=warmup_steps,
    total_steps=total_steps
)

# Compile with Focal Loss
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=lr_schedule),
    loss=FocalLoss(
        gamma=CONFIG['focal_gamma'],
        alpha=CONFIG['focal_alpha'],
        label_smoothing=CONFIG['label_smoothing']
    ),
    metrics=[
        tf.keras.metrics.CategoricalAccuracy(name='accuracy'),
        tf.keras.metrics.TopKCategoricalAccuracy(k=2, name='top2_accuracy')
    ]
)

print("\n" + "="*60)
print("MODEL COMPILED")
print("="*60)
print(f"Optimizer: Adam with Warmup + Cosine Decay")
print(f"Initial LR: {CONFIG['initial_lr']}")
print(f"Warmup epochs: 5")
print(f"Loss: Focal Loss (gamma={CONFIG['focal_gamma']})")
print(f"Label Smoothing: {CONFIG['label_smoothing']}")
print(f"Total steps: {total_steps}")

## 9. Setup Callbacks

In [ ]:
callbacks = [
    # Early stopping - monitor val_accuracy, not loss (because of focal loss)
    tf.keras.callbacks.EarlyStopping(
        monitor='val_accuracy',
        patience=CONFIG['early_stopping_patience'],
        restore_best_weights=True,
        mode='max',
        verbose=1
    ),

    # Reduce LR on plateau
    tf.keras.callbacks.ReduceLROnPlateau(
        monitor='val_accuracy',
        factor=CONFIG['reduce_lr_factor'],
        patience=CONFIG['reduce_lr_patience'],
        min_lr=CONFIG['min_lr'],
        mode='max',
        verbose=1
    ),

    # Save best model
    tf.keras.callbacks.ModelCheckpoint(
        filepath=f"{CONFIG['model_save_dir']}/best_model_v6.keras",
        monitor='val_accuracy',
        save_best_only=True,
        mode='max',
        verbose=1
    ),

    # CSV logger
    tf.keras.callbacks.CSVLogger(
        f"{CONFIG['model_save_dir']}/training_log_v6.csv"
    )
]

print("Callbacks configured:")
for cb in callbacks:
    print(f"  - {cb.__class__.__name__}")

## 10. Train Model

In [ ]:
print("\n" + "="*60)
print("STARTING TRAINING")
print("="*60)
print(f"Epochs: {CONFIG['epochs']} (with early stopping patience {CONFIG['early_stopping_patience']})")
print(f"Batch size: {CONFIG['batch_size']}")
print(f"Training samples: ~{total_train}")
print(f"Validation samples: ~{total_val}")
print(f"Class weights: SQUARED (very strong)")
print(f"Loss: Focal Loss")
print("="*60 + "\n")

# Train!
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=CONFIG['epochs'],
    callbacks=callbacks,
    class_weight=class_weights,
    verbose=1
)

print("\n" + "="*60)
print("TRAINING COMPLETED")
print("="*60)

## 11. Visualize Training Results

In [ ]:
# Plot training history
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# Accuracy
axes[0].plot(history.history['accuracy'], label='Train', marker='o', markersize=2)
axes[0].plot(history.history['val_accuracy'], label='Validation', marker='s', markersize=2)
axes[0].set_title('Model Accuracy', fontsize=14)
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Accuracy')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Loss
axes[1].plot(history.history['loss'], label='Train', marker='o', markersize=2)
axes[1].plot(history.history['val_loss'], label='Validation', marker='s', markersize=2)
axes[1].set_title('Model Loss (Focal)', fontsize=14)
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Loss')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

# Top-2 Accuracy
axes[2].plot(history.history['top2_accuracy'], label='Train', marker='o', markersize=2)
axes[2].plot(history.history['val_top2_accuracy'], label='Validation', marker='s', markersize=2)
axes[2].set_title('Top-2 Accuracy', fontsize=14)
axes[2].set_xlabel('Epoch')
axes[2].set_ylabel('Top-2 Accuracy')
axes[2].legend()
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Print metrics
final_train_acc = history.history['accuracy'][-1]
final_val_acc = history.history['val_accuracy'][-1]
best_val_acc = max(history.history['val_accuracy'])
best_epoch = history.history['val_accuracy'].index(best_val_acc) + 1
overfitting_gap = abs(final_train_acc - final_val_acc)

print("\n" + "="*60)
print("FINAL TRAINING METRICS")
print("="*60)
print(f"Final Train Accuracy: {final_train_acc:.4f} ({final_train_acc*100:.2f}%)")
print(f"Final Val Accuracy:   {final_val_acc:.4f} ({final_val_acc*100:.2f}%)")
print(f"Best Val Accuracy:    {best_val_acc:.4f} ({best_val_acc*100:.2f}%)")
print(f"Best Epoch:           {best_epoch}")
print(f"Overfitting Gap:      {overfitting_gap:.4f} ({overfitting_gap*100:.2f}%)")

if overfitting_gap < 0.05:
    print("\nExcellent generalization!")
elif overfitting_gap < 0.10:
    print("\nGood generalization")
elif overfitting_gap < 0.15:
    print("\nModerate overfitting")
else:
    print("\nHigh overfitting - consider more regularization")

## 12. Load Best Model & Evaluate

In [ ]:
# Load best model
best_model = tf.keras.models.load_model(
    f"{CONFIG['model_save_dir']}/best_model_v6.keras",
    custom_objects={'FocalLoss': FocalLoss}
)
print("Best model loaded")

# Evaluate
print("\nEvaluating on validation set...")
results = best_model.evaluate(val_ds, verbose=1)

print("\n" + "="*60)
print("BEST MODEL PERFORMANCE")
print("="*60)
print(f"Validation Loss:      {results[0]:.4f}")
print(f"Validation Accuracy:  {results[1]:.4f} ({results[1]*100:.2f}%)")
print(f"Validation Top-2 Acc: {results[2]:.4f} ({results[2]*100:.2f}%)")

## 13. Detailed Evaluation (CRITICAL: Check Class Distribution)

In [ ]:
# Get predictions
print("Generating predictions...")

# Reload validation set for predictions
val_ds_eval = tf.keras.utils.image_dataset_from_directory(
    CONFIG['data_dir'],
    validation_split=CONFIG['validation_split'],
    subset="validation",
    seed=CONFIG['seed'],
    image_size=(CONFIG['img_height'], CONFIG['img_width']),
    batch_size=CONFIG['batch_size'],
    label_mode='int',
    shuffle=False
)

# Preprocess
val_ds_eval = val_ds_eval.map(
    lambda x, y: (preprocess_input(x), y),
    num_parallel_calls=AUTOTUNE
)

y_true = []
y_pred = []
y_pred_proba = []

for images, labels in val_ds_eval:
    predictions = best_model.predict(images, verbose=0)
    y_true.extend(labels.numpy())
    y_pred.extend(np.argmax(predictions, axis=1))
    y_pred_proba.extend(predictions)

y_true = np.array(y_true)
y_pred = np.array(y_pred)
y_pred_proba = np.array(y_pred_proba)

accuracy = np.mean(y_true == y_pred)
print(f"\nPredictions generated")
print(f"Total: {len(y_pred)}")
print(f"Correct: {np.sum(y_true == y_pred)}")
print(f"Accuracy: {accuracy:.4f} ({accuracy*100:.2f}%)")

# Check prediction distribution - CRITICAL!
print("\n" + "="*60)
print("PREDICTION DISTRIBUTION (Check for Class Collapse!)")
print("="*60)
pred_counts = {name: 0 for name in class_names}
for p in y_pred:
    pred_counts[class_names[p]] += 1

collapse_detected = False
for class_name, count in pred_counts.items():
    pct = count/len(y_pred)*100
    status = "WARNING - Possible collapse!" if pct > 50 else ""
    if pct > 50:
        collapse_detected = True
    print(f"  {class_name}: {count} predictions ({pct:.1f}%) {status}")

if collapse_detected:
    print("\nClass collapse detected! Model is biased.")
else:
    print("\nPredictions are distributed across classes.")

In [ ]:
# Confusion matrix
cm = confusion_matrix(y_true, y_pred)

plt.figure(figsize=(10, 8))
sns.heatmap(
    cm,
    annot=True,
    fmt='d',
    cmap='Blues',
    xticklabels=class_names,
    yticklabels=class_names,
    square=True,
    linewidths=0.5
)
plt.title('Confusion Matrix - Dr Green V6', fontsize=16, pad=20)
plt.xlabel('Predicted', fontsize=12)
plt.ylabel('True', fontsize=12)
plt.tight_layout()
plt.show()

# Classification report
print("\n" + "="*60)
print("CLASSIFICATION REPORT")
print("="*60)
print(classification_report(y_true, y_pred, target_names=class_names, digits=4))

# Per-class accuracy
print("\nPer-class accuracy:")
for i, class_name in enumerate(class_names):
    class_mask = y_true == i
    if class_mask.sum() > 0:
        class_acc = (y_pred[class_mask] == i).mean()
        emoji = "OK" if class_acc >= 0.60 else "LOW" if class_acc >= 0.40 else "BAD"
        print(f"  [{emoji}] {class_name:12s}: {class_acc:.4f} ({class_acc*100:.2f}%) - {class_mask.sum()} samples")
    else:
        print(f"  [N/A] {class_name:12s}: No samples in validation")

## 14. Save Final Model

In [ ]:
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
model_name = f"drgreen_v6_smalldata_{timestamp}"

print("\n" + "="*60)
print(f"SAVING MODEL: {model_name}")
print("="*60)

# 1. Keras format
keras_path = f"{CONFIG['model_save_dir']}/{model_name}.keras"
best_model.save(keras_path)
print(f"Keras: {keras_path}")

# 2. TFLite (optimized for mobile)
converter = tf.lite.TFLiteConverter.from_keras_model(best_model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()

tflite_path = f"{CONFIG['model_save_dir']}/{model_name}.tflite"
with open(tflite_path, 'wb') as f:
    f.write(tflite_model)
print(f"TFLite: {tflite_path}")

# 3. Metadata
metadata = {
    'model_name': model_name,
    'version': '6.0-smalldata',
    'created_at': timestamp,
    'approach': 'MobileNetV2 frozen + Focal Loss + Strong regularization',
    'config': CONFIG,
    'class_names': class_names,
    'performance': {
        'val_accuracy': float(results[1]),
        'val_top2_accuracy': float(results[2]),
        'val_loss': float(results[0]),
        'best_val_accuracy': float(best_val_acc),
        'overfitting_gap': float(overfitting_gap)
    }
}

metadata_path = f"{CONFIG['model_save_dir']}/{model_name}_metadata.json"
with open(metadata_path, 'w') as f:
    json.dump(metadata, f, indent=2)
print(f"Metadata: {metadata_path}")

# 4. Class names
class_names_path = f"{CONFIG['model_save_dir']}/class_names.json"
with open(class_names_path, 'w') as f:
    json.dump(class_names, f, indent=2)
print(f"Class names: {class_names_path}")

# File sizes
keras_size = os.path.getsize(keras_path) / (1024*1024)
tflite_size = os.path.getsize(tflite_path) / (1024*1024)
print(f"\nFile sizes:")
print(f"  Keras:  {keras_size:.2f} MB")
print(f"  TFLite: {tflite_size:.2f} MB")

## 15. Final Summary

In [ ]:
print("\n" + "="*60)
print("DR GREEN V6 - TRAINING COMPLETE")
print("="*60)

print(f"\nArchitecture:")
print(f"  Base: {CONFIG['base_model']} (FROZEN)")
print(f"  Parameters: {model.count_params():,}")
print(f"  Trainable: {trainable:,}")

print(f"\nPerformance:")
print(f"  Validation Accuracy: {results[1]*100:.2f}%")
print(f"  Top-2 Accuracy:      {results[2]*100:.2f}%")
print(f"  Overfitting Gap:     {overfitting_gap*100:.2f}%")

print(f"\nPrediction Distribution:")
for class_name, count in pred_counts.items():
    pct = count/len(y_pred)*100
    print(f"  {class_name}: {pct:.1f}%")

print(f"\nSaved:")
print(f"  {model_name}.keras")
print(f"  {model_name}.tflite")

if not collapse_detected:
    print("\n" + "="*60)
    print("READY FOR DEPLOYMENT!")
    print("="*60)
else:
    print("\n" + "="*60)
    print("CLASS COLLAPSE DETECTED - Need different approach")
    print("="*60)

## 16. Download Models (Colab)

In [ ]:
# Download models from Colab
from google.colab import files

print("Downloading models...")
print("\nClick the links below to download:")

# Download Keras model
files.download(keras_path)

# Download TFLite model
files.download(tflite_path)

# Download metadata
files.download(metadata_path)

# Download class names
files.download(class_names_path)

print("\nDownloads initiated!")